TO-DO

Create project class
- Create internal, prescraped, external attrs

Import code:
- Add project folder option
- Add check if files exist
- Add confirmation message

Export code:
- Add check if project folder exists and if files exist
- Add confirmation message

Replace code to use these attrs

In [93]:
# Install Requests
# Library requried to get retrieve data from websites
!pip install requests

In [94]:
# Install BeautifulSoup
# HTML parsing library to help with data extraction
!pip install beautifulsoup4

In [127]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import re

In [128]:
# Add User-Agent parameter
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [129]:
# Retrieve all links from a given webpage
# Separate links into internal and external links
def getlinks(url, siteurl):
    
    # Retrieve content from url
    page_link = url
    page_response = requests.get(page_link, headers=headers, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser",
                                 from_encoding="iso-8859-1")
    
    # Create sets for internal and external links
    internal = set([])
    external = set([])
    
    # Retrieve links and sorts them into internal and external links
    links = set([])
    for unsorted_link in page_content.findAll(
        'a', attrs={'href': re.compile("^http")}):
        link = unsorted_link.get('href')
        if link.startswith(siteurl):
            internal.add(link)
        else:
            external.add(link)
            
    return internal, external

In [130]:
# Setting starting variables for sitecrawling
# Given a siteurl, this bot aims to crawl through the site
# and return all internal and external sites linked within
# in two sets respectively

# Initialise sets
sites = set([])
external_sites = set([])

# Set the siteurl for crawling
#siteurl = 'http://cheersingapore.org/cas/'
siteurl = 'https://noc.com.sg/foodking/'

# Initialise links to iterate over for next iteration
links_to_iter = set([siteurl])

In [131]:
# Function to import previous site-link scraping project
def import_sites():
    
    # Import internal_sites
    with open('internal_sites.csv', mode = 'r') as csv_load:
        reader = csv.reader(csv_load)
        for site in reader:
            sites.add(site[0])
            
    # Import prescraped_sites (Resets links_to_iter)
    links_to_iter.clear()
    with open('prescraped_sites.csv', mode = 'r') as csv_load:
        reader = csv.reader(csv_load)
        for site in reader:
            links_to_iter.add(site[0])
    
    # Import external_sites
    with open('external_sites.csv', mode = 'r') as csv_load:
        reader = csv.reader(csv_load)
        for site in reader:
            external_sites.add(site[0])

In [132]:
# Check if user wants to import information
if input('\nImport previous progress? (y/n) ') != 'n':
    import_sites()
    print('\nSites imported.')
    print(f'{len(sites)} internal sites imported')
    print(f'{len(links_to_iter)} prescraped sites imported')
    print(f'{len(external_sites)} external sites imported')
else:
    print('\nSites not imported')


Import previous progress? (y/n) n

Sites not imported


In [133]:
# Iterate through siteurl, returning all internal and external links
# All links are check for duplicates within and against the sets
# New internal links are grouped and all internal links will be
# added to the next iteration

# Start iteration
num_iter = 1
max_iter = 5
continuous_iter = False
has_new_links = True
while has_new_links and num_iter <= max_iter:
    
    # Initialise set for new links
    new_links_group = set([])
    print('Iteration:', num_iter)
    
    # Setup iteration information
    num_links = len(links_to_iter)
    print(f'\n{num_links} links to scrape')
    link_num = 1
    
    # Iterate over each site
    for site in links_to_iter:
        
        print(f'\nIterating over {site}')
        print(f'Site {link_num}/{num_links}')
        
        internal, external = getlinks(site, siteurl)
        external_sites = external_sites|external
        
        # Skip to next site if no internal links
        if len(internal) == 0:
            link_num = link_num + 1
            continue
        # Check for new internal links and group them up
        new_links = internal.difference(sites)
        new_links_group = new_links_group|new_links
        
        link_num = link_num + 1
    
    num_iter = num_iter + 1
    
    # Add new internal links found in this iteration
    # To the next iteration
    links_to_iter = new_links_group
    # To the internal sites set
    sites = sites|new_links_group
    
    # End the iteration process if no new links are found
    if len(links_to_iter) == 0:
        has_new_links = False
        
        print(f'No new links found in iteration {num_iter-1}.\n')
        
        break
    
    # Check new added links
    print(f'\n New links from iteration {num_iter}... {len(new_links_group)} new links\n')
    for link in new_links_group:
        print(link)
    print('\n')
    
    # Continue with next iteration or stop iterating
    if not continuous_iter and num_iter <= max_iter:
        if input(f'Continue with iteration {num_iter}? (y/n) ') != 'y':
            break
        
print(f'{num_iter - 1} iterations done.\n')

Iteration: 1

1 links to scrape

Iterating over https://noc.com.sg/foodking/
Site 1/1

 New links from iteration 2... 15 new links

https://noc.com.sg/foodking/food-king-the-best-of-the-best-from-marine-parade-hawker-centre/
https://noc.com.sg/foodking/food-king-sentosa-grillfest-2019/
https://noc.com.sg/foodking/page/2/
https://noc.com.sg/foodking/food-king-cheap-and-good-hotpots/
https://noc.com.sg/foodking/food-king-unusual-snacks-you-probably-never-heard-of/
https://noc.com.sg/foodking/food-king-the-best-and-worse-from-japan-food-fair/
https://noc.com.sg/foodking/
https://noc.com.sg/foodking/page/3/
https://noc.com.sg/foodking/food-king-fans-top-halal-recommendations/
https://noc.com.sg/foodking/food-king-best-seafood-ever/
https://noc.com.sg/foodking/page/7/
https://noc.com.sg/foodking/food-king-the-craziest-ice-cream-menu-ever/
https://noc.com.sg/foodking/food-king-soupy-delights-you-cant-miss/
https://noc.com.sg/foodking/food-king-2-meals-in-singapore-too-good-to-be-true/
https:


Iterating over https://noc.com.sg/foodking/food-king-ultimate-hidden-gems/
Site 36/37

Iterating over https://noc.com.sg/foodking/page/6/
Site 37/37

 New links from iteration 4... 33 new links

https://noc.com.sg/foodking/nailed-it-spanish-chorizo-with-aurelia/
https://noc.com.sg/foodking/nailed-it-simple-and-affordable-home-cook-food/
https://noc.com.sg/foodking/what-a-deal-what-a-steal-2/
https://noc.com.sg/foodking/food-king-delightful-desserts/
https://noc.com.sg/foodking/nailed-it-ayam-penyet-burger/
https://noc.com.sg/foodking/food-king-best-hawker-food-in-abc-market/
https://noc.com.sg/foodking/legit-street-food-in-sg/
https://noc.com.sg/foodking/food-king-special-episode-feat-nathan-hartono-from-sing-china/
https://noc.com.sg/foodking/food-king-crazy-22-nasi-lemak/
https://noc.com.sg/foodking/food-king-special-episode-feat-michelle-chong/
https://noc.com.sg/foodking/food-king-make-yishun-great-again/
https://noc.com.sg/foodking/food-king-romantic-dinning-with-a-view/
https://

In [134]:
# Sorting all site links alphabetically
sites = sorted(sites)
links_to_iter = sorted(links_to_iter)
external_sites = sorted(external_sites)

In [135]:
# Function for saving links to files
def export_sites():
    print('\nSaving all internal links to internal_sites.csv')
    import csv
    with open('internal_sites.csv', mode = 'w', newline = '') as csv_save:
        writer = csv.writer(csv_save)
        for site in sites:
            # csv.writer.writerow() expects a list of strings,
            # if a single string is given, it is treated as
            # a list of one-character strings instead
            # therefore [site] is required
            writer.writerow([site])
    print(f'{len(sites)} internal sites saved.')

    print('\nSaving prescraped links to prescraped_sites.csv')
    with open('prescraped_sites.csv', mode = 'w', newline = '') as csv_save:
        writer = csv.writer(csv_save)
        for site in links_to_iter:
            writer.writerow([site])
    print(f'{len(links_to_iter)} prescraped sites saved.')

    print('\nSaving all external links to external_sites.csv')
    with open('external_sites.csv', mode = 'w', newline = '') as csv_save:
        writer = csv.writer(csv_save)
        for site in external_sites:
            writer.writerow([site])
    print(f'{len(external_sites)} external sites saved.')
    
    print('\n')

In [138]:
# Check if internal_sites.csv, prescraped_sites.csv,
# and external_sites.csv files exist

# If it exists, prompt if user wants to overwrite files
import pathlib
path = pathlib.Path('internal_sites.csv')

if path.is_file():
    if input('Site files exist. Overwrite files? (y/n) ') != 'y':
        print('Sites not exported.')
    else:
        export_sites()
        print('Sites exported')
else:
    export_sites()
    print('Sites exported.')

Site files exist. Overwrite files? (y/n) y

Saving all internal links to internal_sites.csv
85 internal sites saved.

Saving prescraped links to prescraped_sites.csv
0 prescraped sites saved.

Saving all external links to external_sites.csv
297 external sites saved.


Sites exported


In [34]:
for site in sites:
    print(site)

http://cheersingapore.org/cas/category/events/page/3/
http://cheersingapore.org/cas/2016/06/13/charm-cheerleading-championships-c3-2016/#respond
http://cheersingapore.org/cas/wp-content/uploads/2013/09/ACU_3990.jpg
http://cheersingapore.org/cas/2015/01/05/casncc-and-acic-2015-competition-date/#respond
http://cheersingapore.org/cas/category/events/page/4/
http://cheersingapore.org/cas/wp-content/uploads/2014/12/National-All-Girl-Team-2014-Second-Tryout-Information.pdf
http://cheersingapore.org/cas/technical/coaches/accreditation-and-pathway/
http://cheersingapore.org/cas/wp-content/uploads/2017/08/Cheerleading-Competition-Release-Indemnity-Form-2017Aug21.pdf
http://cheersingapore.org/cas/national-team/national-team-2017/
http://cheersingapore.org/cas/wp-content/uploads/2014/08/Kenneth-and-Jasmine.jpg
http://cheersingapore.org/cas/wp-content/uploads/2017/01/CASNCC-2017-Information-Package.pdf
http://cheersingapore.org/cas/2017/08/21/mediacorp-cheerleading-competition-2017/
http://cheersi

In [35]:
for site in external_sites:
    print(site)

http://www.straitstimes.com/news/singapore/more-singapore-stories/story/cheerleaders-death-its-time-take-serious-look-cheerleadi
http://www.youtube.com/watch?v=ePhaDWfsM2Q
http://www.cheerleading.org.sg/
http://www.phfslipperrace.org/sg/
http://cheerunion.org/education/chrdarules/
http://www.cheerleading.com.sg/events/competitions/asia-cheerleading-invitational-championships/
http://cheerleading.org.sg/casncc2013
http://www.cheerleading.com.sg/events/competitions/cas-national-cheerleading-championships/participants-list/
http://themezee.com/
http://cheersingapore.org/cas
http://cas@cheerleading.org.sg
http://www.cheerunion.org/news/?article_id=1
http://cheerleading.com.sg/events/downloads/
http://www.youtube.com/watch?v=NGBOppKK25Y
http://cheerunion.org.ismmedia.com/ISM3/std-content/repos/Top/docs/ICU_TeamCheer_Rules_2018.pdf
http://cheerunion.org.ismmedia.com/ISM3/std-content/repos/Top/Championships/ICU_Special-Abilities_Rules_2017.pdf
http://cheersingapore.org/casncc2014
http://www.c

In [ ]:
# Implement progressbar (progressbar2)
# Arrange links in alphabetical order